# Flask & Pydantic: Auto-Server und Client


Dieses Notebook zeigt, wie ein **Flask-Server** mit **Pydantic** Daten überprüft und wie ein **Python-Client** mit dem Modul `requests` Daten an den Server sendet.

Ziel ist es, ein Auto-Objekt zu erstellen und sicherzustellen, dass alle Eingaben gültig sind.


## Server: `handle_auto()`


Der Server empfängt JSON-Daten, erstellt ein `Auto`-Objekt und prüft diese mit **Pydantic**.  
Bei gültigen Daten sendet er eine Erfolgsnachricht (HTTP 201), bei fehlerhaften eine Fehlermeldung (HTTP 400).


In [ ]:

from flask import Flask, request, jsonify
from pydantic import BaseModel, Field, ValidationError

app = Flask(__name__)

# Pydantic-Modell
class Auto(BaseModel):
    marke: str = Field(..., min_length=2, description="Name der Automarke")
    ps: int = Field(..., ge=50, le=1000, description="Leistung in PS (50–1000)")
    verbrauch: float = Field(..., ge=0, le=20, description="Verbrauch in Litern pro 100 km")

    model_config = {"validate_assignment": True}

@app.route("/auto", methods=["POST"])
def handle_auto():
    """Erstellt ein neues Auto aus JSON-Daten und prüft es mit Pydantic."""
    try:
        data = request.get_json()
        auto = Auto(**data)
        return jsonify({
            "status": "ok",
            "message": "Auto erfolgreich erstellt",
            "auto": auto.model_dump()
        }), 201
    except ValidationError as e:
        return jsonify({
            "status": "error",
            "message": "Validierung fehlgeschlagen",
            "details": e.errors()
        }), 400

# Hinweis: Zum Starten außerhalb des Notebooks ausführen:
# if __name__ == "__main__":
#     app.run(debug=True)


## Client: Daten an den Server senden


Der Client sendet über `requests.post()` ein JSON-Objekt an den Endpunkt `/auto`.  
Der Server validiert die Daten und gibt eine Erfolgsmeldung oder einen Fehler zurück.


In [ ]:

import requests
import json

URL = "http://127.0.0.1:5000/auto"

# 🚗 Beispiel-Daten
auto_daten = {
    "marke": "Audi",
    "ps": 150,
    "verbrauch": 6.5
}

response = requests.post(URL, json=auto_daten)
print("Statuscode:", response.status_code)

if response.status_code == 201:
    print("✅ Auto erfolgreich erstellt:")
    print(json.dumps(response.json(), indent=4, ensure_ascii=False))
else:
    print("❌ Fehler bei der Erstellung:")
    print(json.dumps(response.json(), indent=4, ensure_ascii=False))


## ⚠️ Fehlerbeispiele


Hier werden absichtlich falsche Daten geschickt, um zu zeigen, wie Pydantic automatisch eine **ValidationError**-Meldung erzeugt.


In [ ]:

# Ungültige Daten (zu hohe PS und negativer Verbrauch)
fehler_daten = {
    "marke": "VW",
    "ps": 2000,
    "verbrauch": -5
}

response = requests.post(URL, json=fehler_daten)
print("Statuscode:", response.status_code)
print(json.dumps(response.json(), indent=4, ensure_ascii=False))



## 🧠 Fazit

| Schritt | Aufgabe | Werkzeug |
|----------|----------|-----------|
| 1 | Server empfängt Daten | `Flask` |
| 2 | Validierung der Daten | `Pydantic` |
| 3 | Rückgabe bei Erfolg oder Fehler | HTTP 201 / 400 |
| 4 | Client sendet Daten | `requests.post()` |
| 5 | Ergebnis anzeigen | `response.json()` |

➡️ So entsteht ein vollständiger **Client-Server-Datenaustausch** mit **Pydantic-Validierung** am Beispiel eines **Autos**.


## <font color=red >Übung</font> 
1. Erstelle das Modul `client2`

  - Frage die Daten eines Spielers von der Konsole ab und erzeuge daraus ein Spielerobjekt.
  - Übertrage das Spielerobjekt als JSON an den Server.
  - Gib die Antwort des Servers auf der Konsole aus.

2. Erweitere das Modul `server`

  - Erstelle eine neue Route `/spieler` zu einer neuen Methode `handle_spieler`.
  - Erzeuge aus den empfangenen Daten ein Objekt vom Typ Spieler
  - Gib die Daten des Spielers als JSON als Antwort an den Client zurück
  